In [60]:
import os
import glob

def get_latest_files(base_dir):
    latest_files = []

    # Each subfolder corresponds to a cloud
    for cloud_dir in os.listdir(base_dir):
        cloud_path = os.path.join(base_dir, cloud_dir)
        if not os.path.isdir(cloud_path):
            continue

        # Match CSV files in the form: cloudname-*.csv
        pattern = os.path.join(cloud_path, f"{cloud_dir}-*.csv")
        csv_files = glob.glob(pattern)

        if not csv_files:
            continue

        # Pick the most recent file by modified timestamp
        latest_file = max(csv_files, key=os.path.getmtime)
        latest_files.append(latest_file)

    return latest_files

# Example usage:
base_dir = os.path.join("out_tfg_v2", "full")
latest_csvs = get_latest_files(base_dir)

# Output the list (or use it as input for further processing)
for f in latest_csvs:
    print(f)


out_tfg_v2/full/5080_54400/5080_54400-2025-05-13-13:38:18.csv
out_tfg_v2/full/Paris_Luxembourg_6/Paris_Luxembourg_6-2025-05-13-14:13:18.csv
out_tfg_v2/full/bildstein_station1_xyz_intensity_rgb/bildstein_station1_xyz_intensity_rgb-2025-05-13-13:08:25.csv
out_tfg_v2/full/sg27_station8_intensity_rgb/sg27_station8_intensity_rgb-2025-05-13-15:55:35.csv
out_tfg_v2/full/Lille_0/Lille_0-2025-05-13-13:45:39.csv
out_tfg_v2/full/Speulderbos_2017_TLS/Speulderbos_2017_TLS-2025-05-13-20:56:18.csv


In [61]:
import pandas as pd
import os

# Grouping structure: cloud -> list of (octree_type, encoder, mean_time)
rows = []

# Mapping helpers
octree_map = {
    "Octree": "Punteros",
    "LinearOctree": "Lineal"
}

encoder_map = {
    "NoEncoding": "-",
    "MortonEncoder3D": "Morton",
    "HilbertEncoder3D": "Hilbert"
}

# Iterate over CSV files
for filename in latest_csvs:
    cloud = os.path.basename(os.path.dirname(filename))
    cloud = cloud.replace("_", r"\_")
    df = pd.read_csv(filename)
    df = df[df["kernel"] == "Sphere"]

    npoints = df["npoints"].iloc[0]
    radius = df["radius"].max()

    row_group = []
    for octree_type in ["Octree", "LinearOctree"]:
        algo = "neighborsPtr" if octree_type == "Octree" else "neighborsV2"
        
        for encoder_key in ["NoEncoding", "MortonEncoder3D", "HilbertEncoder3D"]:
            if octree_type == "LinearOctree" and encoder_key == "NoEncoding":
                continue
            subset = df[
                (df["octree"] == octree_type) &
                (df["encoder"] == encoder_key) &
                (df["operation"] == algo) &
                (df["radius"] == radius)
            ]
            if subset.shape[0] == 1:
                mean_time = subset["mean"].iloc[0]
                row_group.append((octree_map[octree_type], encoder_map[encoder_key], mean_time))
            else:
                print("WARNING: didnt found correct data for: ", octree_type, encoder_key, algo, radius, cloud)


    rows.append((cloud, npoints, radius, row_group))

# Generate LaTeX
latex = r"""\begin{table*}[ht]
\centering
\caption{Tiempos de ejecución de búsquedas de vecinos sobre todos los puntos de la nube con \textit{Sphere kernel}.}
\label{tab:results-full}
\begin{tabular}{|c|c|c|ccc|}
\hline
Nube                      & $n$                    & $r$ (m)                  & \Octree                    & SFC     & Tiempo (s)\\ \hline
"""

for cloud, npoints, radius, group in rows:
    n = f"{npoints / 1e6:.2f}M"
    r = f"{radius:.1f}"

    latex += f"\\multirow{{5}}{{*}}{{\\textit{{{cloud}}}}} & \\multirow{{5}}{{*}}{{{n}}} & \\multirow{{5}}{{*}}{{{r}}} "

    first = True
    for idx, (octree, encoder, time) in enumerate(group):
        if not first:
            latex += " &  &  "
        else:
            first = False

        if idx == 3:
            latex += r" & \multirow{2}{*}{Lineal} "
        elif idx == 0:
            latex += r" & \multirow{3}{*}{Punteros} "
        else:
            latex += " & "

        latex += f"& {encoder} & {time:.2f} \\\\\n"
        if idx == 2 or idx == 4:
            latex += r"\cline{4-6}" + "\n"

    latex += r"\hline" + "\n"

latex += r"""\end{tabular}
\end{table*}
"""


In [62]:
OUTPUT_TEX = "generated_table.tex"
# Save to file
with open(OUTPUT_TEX, "w") as f:
    f.write(latex)

print(f"LaTeX table written to {OUTPUT_TEX}")

LaTeX table written to generated_table.tex


In [63]:

# Grouping structure: cloud -> list of (algo_type, mean_time)
rows = []

algo_map = {
    "neighborsPtr": r"\dotNeighborsPtr\textit{neighborsPtr}",
    "neighbors": r"\dotNeighbors\textit{neighbors}",
    "neighborsV2": r"\dotNeighborsVtwo\textit{neighborsV2}",
    "neighborsStruct": r"\dotNeighborsStruct\textit{neighborsStruct}"
}

# Iterate over CSV files
for filename in latest_csvs:
    cloud = os.path.basename(os.path.dirname(filename))
    cloud = cloud.replace("_", r"\_")
    df = pd.read_csv(filename)
    df = df[(df["kernel"] == "Sphere") & (df["encoder"] == "MortonEncoder3D")]
    npoints = df["npoints"].iloc[0]
    radius = df["radius"].max()
    row_group = []
    for algo in ["neighborsPtr", "neighbors", "neighborsV2", "neighborsStruct"]:
        subset = df[
            (df["operation"] == algo) &
            (df["radius"] == radius)
        ]
        if subset.shape[0] == 1:
            mean_time = subset["mean"].iloc[0]
            row_group.append((algo_map[algo], mean_time))
        else:
            print("WARNING: didnt found correct data for: ", algo, radius, cloud)
    rows.append((cloud, npoints, radius, row_group))

# Generate LaTeX
latex = r"""\begin{table*}[ht]
\centering
\caption{Tiempos de ejecución de búsquedas de vecinos sobre todos los puntos de la nube con \textit{Sphere kernel}.}
\label{tab:results-full}
\begin{tabular}{|c|c|l|l|}
\hline
Nube & $r$ (m) & Algoritmo & Tiempo (s)\\ \hline
"""

for cloud, npoints, radius, group in rows:
    n = f"{npoints / 1e6:.2f}M"
    r = f"{radius:.1f}"

    latex += f"\\multirow{{4}}{{*}}{{\\textit{{{cloud}}}}} & \\multirow{{4}}{{*}}{{{r}}} "

    first = True
    for idx, (algo, time) in enumerate(group):
        if not first:
            latex += " & "
        else:
            first = False

        latex += f"& {algo} & {time:.2f} \\\\\n"

    latex += r"\hline" + "\n"

latex += r"""\end{tabular}
\end{table*}
"""


In [64]:
OUTPUT_TEX = "generated_table_2.tex"
# Save to file
with open(OUTPUT_TEX, "w") as f:
    f.write(latex)

print(f"LaTeX table written to {OUTPUT_TEX}")

LaTeX table written to generated_table_2.tex
